In [ ]:
# alpha=0.001, beta=0.00575, tEnd=8000
from matplotlib import pyplot as plt
import numpy as np
import math
from scipy.integrate import solve_ivp
from scipy import integrate
from matplotlib.pyplot import figure

def JJ (t, y, xi_0, T_1_, xi_P, A_P, omega_P, A_0, omega_T_, alpha, beta, c, b_0, a, T_2_, g):
    x1, y1, x2, y2, z = y
    dx1 = x1*(1-xi_0*math.e**(-1/T_1_)-xi_P)-y1-1/3*x1**3+A_P*math.cos(omega_P*t)+A_0*math.e**(-1/T_1_)*math.cos(omega_T_*t)-alpha*math.sin(z)-beta*(x1-x2)
    dy1 = c*(-b_0*math.e**(1/T_1_)*y1+a+x1)
    dx2 = x2*(1-xi_0*math.e**(-1/T_2_)-xi_P)-y2-1/3*x2**3+A_P*math.cos(omega_P*t)+A_0*math.e**(-1/T_2_)*math.cos(omega_T_*t)-alpha*math.sin(z)-beta*(x1-x2)
    dy2 = c*(-b_0*math.e**(1/T_2_)*y2+a+x2)
    dz = g*(x1-x2)
    dydt = [dx1, dy1, dx2, dy2, dz]
    return dydt


xi_0 = 0.175
T_1_ = 5
xi_P = 0.175
A_P = 0.01
omega_P = 0.7
A_0 = 0.5
omega_T_ = 0.7
alpha = 0.001 #original: 0.02
beta = 0.00575 #original: 0.02
c = 0.1
b_0 = 0.6
a = 0.7
T_2_ = 5
g = 0.1


tStart = 0
tEnd = 8000

X0 = [0.2, 0.1, 0.02, 0.01, 0.001]
solution = integrate.solve_ivp(JJ, [tStart, tEnd], X0, method='RK45', t_eval=np.linspace(tStart,tEnd,100000), args=(xi_0, T_1_, xi_P, A_P, omega_P, A_0, omega_T_, alpha, beta, c, b_0, a, T_2_, g))

t_sol = solution.t
x1_sol = solution.y[0]
y1_sol = solution.y[1]
x2_sol = solution.y[2]
y2_sol = solution.y[3]
z_sol = solution.y[4]

# error function
error_func = []
i = 0
while i<len(t_sol):
    error_func.append(math.sqrt((x1_sol[i]-x2_sol[i])**2+(y1_sol[i]-y2_sol[i])**2))
    i += 1


figure(figsize=(120, 8))
plt.plot(t_sol, error_func)
plt.xlabel('t', fontsize=60)
plt.ylabel(''r'$\theta$', fontsize=60)
plt.title(''r'$\alpha={0}$, 'r'$\beta={1}$'.format(alpha, beta), fontsize=60)
plt.show()

import ordpy
import pandas as pd
import array

ordinal_distribution = ordpy.ordinal_distribution(error_func, dx=4, dy=1, taux=1, tauy=1, return_missing=False, tie_precision=None)
# print(ordinal_distribution)


ordinal_patterns=[]
probabilities=[]
i=0
while i<len(ordinal_distribution[0]):
    ordinal_patterns.append(ordinal_distribution[0][i])
    probabilities.append(ordinal_distribution[1][i])
    i+=1

# print(ordinal_patterns)
# print(probabilities)

data={"ordinal patterns":ordinal_patterns, "probabilities":probabilities}

dataframe = pd.DataFrame(data)
print(dataframe)

def my_autopct(pct):
    return ('%.2f' % pct) if pct > 1 else ''

fig = plt.figure(figsize =(10, 7))
plt.pie(probabilities, autopct=my_autopct, labels = ordinal_patterns)
plt.legend(ordinal_patterns, loc="center right")